# Genetic Algorithm Optimization for 2048 Heuristic Weights

This notebook demonstrates how to use the Genetic Algorithm (GA) to optimize heuristic weights for the Expectimax agent.

## Overview

The GA evolves a population of genomes, where each genome represents a set of heuristic weights. Each genome is evaluated by playing multiple games with Expectimax using those weights, and fitness is computed from the game results.


## 1. Imports and Setup


In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add parent directory to path for imports
sys.path.insert(0, str(Path().resolve().parent))

from ga import GARunner, Genome, MeanScoreFitness, ScorePlusBonusFitness
from agents.expectimax import ExpectimaxAgent
from game_2048.game_env import GameEnv


ImportError: cannot import name 'GameSummary' from partially initialized module 'stats_logging' (most likely due to a circular import) (C:\Users\milleraa\OneDrive - Milwaukee School of Engineering\Documents\GitHub\2048-agents\stats_logging\__init__.py)

: 

## 2. Configuration

Configure GA parameters. Start with smaller values for testing, then scale up for full optimization.


In [ ]:
# GA Configuration
config = {
    "population_size": 30,  # Start small for testing, increase to 50-100 for real runs
    "num_generations": 20,  # Start small for testing, increase to 50-200 for real runs
    "mutation_rate": 0.1,
    "mutation_strength": 0.1,
    "crossover_rate": 0.7,
    "elite_size": 5,
    "games_per_genome": 10,  # Number of games to evaluate each genome
    "board_size": 4,
    "depth_limit": 5,  # Expectimax depth limit
    "time_limit_ms": None,  # Optional time limit per move
    "seed": 42,  # For reproducibility
    "output_dir": "data/ga_results"
}

# Fitness strategy: "mean_score" or "score_plus_bonus"
fitness_strategy_name = "mean_score"

# Optional: Load initial weights from a previous run
initial_weights_path = None  # e.g., "data/ga_results/best_genome_latest.json"

print("GA Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")


## 3. Initialize Fitness Strategy


In [ ]:
if fitness_strategy_name == "mean_score":
    fitness_strategy = MeanScoreFitness()
elif fitness_strategy_name == "score_plus_bonus":
    fitness_strategy = ScorePlusBonusFitness(
        bonus_2048=10000.0,
        bonus_4096=50000.0,
        bonus_8192=200000.0
    )
else:
    raise ValueError(f"Unknown fitness strategy: {fitness_strategy_name}")

print(f"Using fitness strategy: {fitness_strategy.get_name()}")


## 4. Load Initial Weights (Optional)

If you want to start from a previous best genome, load it here.


In [ ]:
initial_weights = None
if initial_weights_path:
    with open(initial_weights_path, 'r') as f:
        data = json.load(f)
        if 'weights' in data:
            initial_weights = data['weights']
        else:
            initial_weights = data
    print(f"Loaded initial weights from {initial_weights_path}")
    print("Initial weights:")
    for name, weight in initial_weights.items():
        print(f"  {name}: {weight:.4f}")
else:
    print("Starting with random weights")


## 5. Create and Run GA


In [ ]:
# Create GA runner
runner = GARunner(
    population_size=config["population_size"],
    num_generations=config["num_generations"],
    mutation_rate=config["mutation_rate"],
    mutation_strength=config["mutation_strength"],
    crossover_rate=config["crossover_rate"],
    elite_size=config["elite_size"],
    games_per_genome=config["games_per_genome"],
    fitness_strategy=fitness_strategy,
    initial_weights=initial_weights,
    board_size=config["board_size"],
    depth_limit=config["depth_limit"],
    time_limit_ms=config["time_limit_ms"],
    seed=config["seed"],
    output_dir=config["output_dir"]
)

print("Starting GA evolution...")
print("=" * 60)


In [ ]:
# Run evolution
final_population = runner.run()


## 6. Results Summary


In [ ]:
print("=" * 60)
print("Evolution Complete!")
print("=" * 60)
print(f"Best fitness: {runner.best_fitness:.2f}")
print("\nBest weights:")
if runner.best_genome:
    for name, weight in sorted(runner.best_genome.weights.items()):
        print(f"  {name:20s}: {weight:10.4f}")
print(f"\nBest genome saved to: {config['output_dir']}/best_genome_latest.json")


## 7. Visualize Evolution Progress

Load saved genomes to visualize fitness progression across generations.


In [ ]:
# Load all saved genomes
output_dir = Path(config["output_dir"])
genome_files = sorted(output_dir.glob("best_genome_gen_*.json"))

if genome_files:
    generations = []
    fitnesses = []

    for file in genome_files:
        with open(file, 'r') as f:
            data = json.load(f)
            generations.append(data['generation'])
            fitnesses.append(data['fitness'])

    # Plot fitness over generations
    plt.figure(figsize=(10, 6))
    plt.plot(generations, fitnesses, marker='o', linewidth=2, markersize=6)
    plt.xlabel('Generation', fontsize=12)
    plt.ylabel('Best Fitness', fontsize=12)
    plt.title('GA Evolution: Best Fitness Over Generations', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    print(f"Fitness improved from {fitnesses[0]:.2f} to {fitnesses[-1]:.2f}")
    print(f"Improvement: {fitnesses[-1] - fitnesses[0]:.2f} ({((fitnesses[-1] / fitnesses[0] - 1) * 100):.1f}%)")
else:
    print("No saved genome files found for visualization")


## 8. Test Best Weights

Test the best weights by running some games and comparing performance.


In [ ]:
if runner.best_genome:
    print("Testing best weights...")

    # Create agent with best weights
    best_agent = ExpectimaxAgent(
        depth_limit=config["depth_limit"],
        time_limit_ms=config["time_limit_ms"],
        heuristic_weights=runner.best_genome.weights,
        use_iterative_deepening=True
    )

    # Run a few test games
    num_test_games = 5
    scores = []
    highest_tiles = []

    for i in range(num_test_games):
        env = GameEnv(board_size=config["board_size"], seed=config["seed"] + i if config["seed"] else None)
        board, _ = env.reset()

        score = 0
        step_count = 0
        done = False

        while not done:
            legal_moves = env.legal_moves(board)
            if not legal_moves:
                done = True
                break

            action = best_agent.choose_action(board, legal_moves)
            board, reward, done, _ = env.step(action)
            score += int(reward)
            step_count += 1

        highest_tile = int(board.array.max())
        scores.append(score)
        highest_tiles.append(highest_tile)
        print(f"Game {i+1}: Score={score}, Highest Tile={highest_tile}, Steps={step_count}")

    print(f"\nTest Results ({num_test_games} games):")
    print(f"  Mean Score: {np.mean(scores):.2f}")
    print(f"  Std Score: {np.std(scores):.2f}")
    print(f"  Mean Highest Tile: {np.mean(highest_tiles):.2f}")
    print(f"  Games Reached 2048: {sum(1 for t in highest_tiles if t >= 2048)}")
    print(f"  Games Reached 4096: {sum(1 for t in highest_tiles if t >= 4096)}")
else:
    print("No best genome available")


## 9. Compare with Default Weights

Compare the GA-optimized weights with the default Expectimax weights.


In [ ]:
from agents.expectimax import DEFAULT_WEIGHTS

if runner.best_genome:
    print("Weight Comparison:")
    print("=" * 60)
    print(f"{'Feature':<20} {'Default':>12} {'GA Optimized':>15} {'Difference':>12}")
    print("-" * 60)

    all_features = set(DEFAULT_WEIGHTS.keys()) | set(runner.best_genome.weights.keys())

    for feature in sorted(all_features):
        default = DEFAULT_WEIGHTS.get(feature, 0.0)
        optimized = runner.best_genome.weights.get(feature, 0.0)
        diff = optimized - default
        print(f"{feature:<20} {default:12.4f} {optimized:15.4f} {diff:12.4f}")

    print("=" * 60)


## 10. Save Results Summary

Save a summary of the optimization run.


In [ ]:
if runner.best_genome:
    summary = {
        "config": config,
        "fitness_strategy": fitness_strategy_name,
        "best_fitness": runner.best_fitness,
        "best_weights": runner.best_genome.weights,
        "default_weights": DEFAULT_WEIGHTS
    }

    summary_path = Path(config["output_dir"]) / "optimization_summary.json"
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)

    print(f"Summary saved to: {summary_path}")
